In [2]:
print('test')

test


Step 1: Prepare Documents

In [3]:
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

Step 2: Create embeddings using pretrained models

In [4]:
# pip install sentence_transformers==2.7.0

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-mpnet-base-v2")

/Users/wesmacdonald/.pyenv/versions/3.10.13/envs/llm-zoomcamp-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
len(model.encode("This is a simple sentence"))

768

In [10]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

In [11]:
operations[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.03570355847477913,
  -0.06891412287950516,
  -0.044483352452516556,
  0.0062501393258571625,
  -0.042247142642736435,
  -0.006454026326537132,
  0.02754812128841877,
  -0.018610356375575066,
  -0.052037592977285385,
  -0.0024046804755926132,
  0.057454537600278854,
  -0.015592586249113083,
  0.015488773584365845,
  0.09870164841413498,
  0.015649642795324326,


Step 3: Setup ES connection

In [2]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '1de2ed383b2e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'es1hSOCoRA2p8xOYOJuvnw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

Step 4: Creating Mappings & Index

In [14]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector":{"type":"dense_vector","dims": 768,"index":True,"similarity": "cosine"
        },
        }
    }
}

In [8]:
index_name = "course-questions"

In [15]:
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

Step 5: Add documents into index

In [16]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

Step 6: Create end user query

In [5]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [6]:
query = {
    "field" : "text_vector",
    "query_vector" : vector_search_term,
    "k" : 5,
    "num_candidates" : 10000
}

In [9]:
res = es_client.search(index=index_name, knn=query,source=["text","section","question","course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'HY5-epABG2Gp-HVpWVFr',
  '_score': 0.71479183,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'MI5-epABG2Gp-HVpcFQH',
  '_score': 0.61347365,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares 

Step 7: Perform Semantic search & Advanced search 
+ 3.2.2 - Advanced Semantic Search

In [23]:
knn_query = {
    "field" : "text_vector",
    "query_vector" : vector_search_term,
    "k" : 5,
    "num_candidates" : 10000
}

response = es_client.search(
    index=index_name,
    query={
        "match": {
                "section": "General course-related questions"
                # "course" : "machine-learning-zoomcamp"
            },
    },
    knn=knn_query,
    size=5,
    explain=True
)

In [24]:
response["hits"]["hits"]

[{'_shard': '[course-questions][0]',
  '_node': 'REkay9luQtOYvZuPWF2sLA',
  '_index': 'course-questions',
  '_id': 'HY5-epABG2Gp-HVpWVFr',
  '_score': 11.614713,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.026965495198965073,
    -0.0006259257206693292,
    -0.016629504039883614,
    0.05285143107175827,
    0.05476531386375427,
    -0.03133983165025711,
    0.029942642897367477,
    -0.04808562621474266,
    0.04467548429965973,
    0.005839455407112837,
    0.016233131289482117,
    0.012001212686300278,
    -0.0312223918735981,
    0.01660064235329628,
    -0.04886900633573532,
    -0.06496303528547287,
    0.046434178948402405,
    -0.009297648444771767,
    -0.0642528235912323,
    -0.01373270619660616,
 